#Submitted by:

Chigozie Kenneth Okafor 225983

Md Khamar Uz Zama 226267

Rajatha Nagaraja Rao 223758


In [0]:
!python prepare_data.py skp_vocab.txt skp

2020-05-26 09:09:03.873652: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [0]:
from prepare_data import parse_seq
import pickle
import tensorflow as tf

filename = "WandB.pickle"


# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
sequenceLength = 200
data = data.map(lambda x: parse_seq(x, sequenceLength))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)




{'w': 1, 'j': 2, 'i': 3, 'S': 4, 'P': 5, 'W': 6, 'b': 7, 'T': 8, 'V': 9, '.': 10, 'I': 11, 'D': 12, 'v': 13, 'K': 14, '!': 15, 'g': 16, 'p': 17, ']': 18, 'e': 19, '3': 20, 'm': 21, 'y': 22, 'G': 23, 'c': 24, '&': 25, 'h': 26, 'U': 27, 'Q': 28, 'l': 29, 'H': 30, 'n': 31, 'a': 32, 'F': 33, 'r': 34, '[': 35, 'z': 36, ':': 37, 'O': 38, '$': 39, 'u': 40, 'f': 41, ',': 42, 'R': 43, 'L': 44, 'd': 45, '?': 46, 'C': 47, 'Z': 48, ' ': 49, 'A': 50, "'": 51, 'E': 52, 't': 53, 'B': 54, 'x': 55, 'J': 56, 'Y': 57, '-': 58, 'q': 59, 'M': 60, 's': 61, 'X': 62, '\n': 63, ';': 64, 'k': 65, 'N': 66, 'o': 67, '<S>': 0}
68


In [0]:
batch_size = 64
batch_OneHot_data = data.shuffle(buffer_size = 10 * batch_size).batch(batch_size=batch_size, drop_remainder=True).repeat()
batch_size = tf.convert_to_tensor(batch_size, dtype=None, dtype_hint=None, name=None)
print(batch_size)

tf.Tensor(64, shape=(), dtype=int32)


In [0]:
def initializer(hunits, vocab_size, sequenceLength):
    
    
    # Input to hidden layer
    W_InpToHidden = tf.Variable(tf.initializers.glorot_uniform()(shape=[vocab_size, hunits]))
    b_InpToHidden = tf.Variable(tf.initializers.glorot_uniform()(shape=[hunits]))
    
    # hidden to hidden layer
    W_HiddenToHidden = tf.Variable(tf.initializers.glorot_uniform()(shape=[hunits,hunits]))

    
    # hidden to output layer
    W_HiddenToOutput = tf.Variable(tf.initializers.glorot_uniform()(shape=[hunits,vocab_size]))
    b_HiddenToOutput = tf.Variable(tf.initializers.glorot_uniform()(shape=[vocab_size]))
    
    trainVariables = [W_InpToHidden, b_InpToHidden, W_HiddenToHidden, W_HiddenToOutput, b_HiddenToOutput]
    return trainVariables

In [0]:
units = 512
trainVariables = initializer(units, vocab_size, sequenceLength)

for value in trainVariables:
    print(value.shape)
    
    


(68, 512)
(512,)
(512, 512)
(512, 68)
(68,)


In [0]:

opt = tf.optimizers.Adam()
n_time_steps = 20

In [0]:
#All operations were executed using tensorflow as python operations do not work under tensorflow

# @tf.function
def RNNCell(seq):
  with tf.GradientTape() as tape:
    state = tf.zeros([batch_size, units])
    


    for i in tf.range(n_time_steps-1):
        inp = tf.one_hot(seq[:, i], vocab_size)
        # # print("inp:  ", inp.shape)
        op_InpToHidden = tf.matmul(inp, trainVariables[0])
        # # print("op_InpToHidden :  ", op_InpToHidden.shape) 
        op_HiddenToHidden = tf.matmul(state, trainVariables[2]) + trainVariables[1] 
        # # print("op_HiddenToHidden :  ", op_HiddenToHidden.shape)
        op_FromHidden = op_InpToHidden + op_HiddenToHidden
        # # print("op_FromHidden :  ", op_FromHidden.shape)
        state = tf.nn.tanh(op_FromHidden)
        # print("op_FromHidden :  ", op_FromHidden.shape)


        # Second node
        y = tf.matmul(state, trainVariables[3]) + trainVariables[4]
        
        # print(y)

        labels = seqs[:, i+1]
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels, y)
        loss = tf.reduce_mean(loss)

        train_perplexity = tf.exp(loss)

        accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        acc = accuracy(labels, y)



    grads = tape.gradient(loss, trainVariables)

    
    opt.apply_gradients(zip(grads, trainVariables))

    return loss, acc, train_perplexity





In [ ]:
def trainModel():
    steps = 10000
    
    for step, seqs in enumerate(batch_OneHot_data):
    
        loss, acc= RNNCell(seqs)
    
        if not step % 100:
            print("Step: {} Loss: {} Acc: {}\n".format(step, loss, acc))
    
        if step > steps:
            break


In [0]:
try:
    with open('WandB.pickle', 'rb') as f:
        trainVariables = pickle.load(f)
except:        
    trainVariables = initializer(units, vocab_size, sequenceLength)
    trainModel()

Step: 0 Loss: 4.187745571136475 ACC: 0.015625 Perplexity: 65.87411499023438 

Step: 100 Loss: 3.315243721008301 ACC: 0.171875 Perplexity: 27.529102325439453 

Step: 200 Loss: 2.7459614276885986 ACC: 0.296875 Perplexity: 15.579585075378418 

Step: 300 Loss: 2.98148250579834 ACC: 0.1875 Perplexity: 19.717025756835938 

Step: 400 Loss: 2.7067885398864746 ACC: 0.3125 Perplexity: 14.981085777282715 

Step: 500 Loss: 2.5962560176849365 ACC: 0.28125 Perplexity: 13.413423538208008 

Step: 600 Loss: 2.499633312225342 ACC: 0.3125 Perplexity: 12.178028106689453 

Step: 700 Loss: 2.532416820526123 ACC: 0.359375 Perplexity: 12.583882331848145 

Step: 800 Loss: 2.3146772384643555 ACC: 0.3125 Perplexity: 10.121655464172363 

Step: 900 Loss: 2.4612746238708496 ACC: 0.34375 Perplexity: 11.71973991394043 

Step: 1000 Loss: 2.3218870162963867 ACC: 0.25 Perplexity: 10.194893836975098 

Step: 1100 Loss: 2.4397318363189697 ACC: 0.265625 Perplexity: 11.469964981079102 

Step: 1200 Loss: 2.607088088989258 ACC

In [0]:
with open(filename, 'wb') as f:
   pickle.dump(trainVariables,f)

In [0]:
import numpy as np

def generate(steps):
    sentence = ""
    state = tf.zeros([1, units])
    seq = [0]

    for step in range(steps):

        inp = tf.one_hot(seq[-1: ], vocab_size)
        # print("inp:  ", inp.shape)
        op_InpToHidden = tf.matmul(inp, trainVariables[0])
        # print("op_InpToHidden :  ", op_InpToHidden.shape) 
        op_HiddenToHidden = tf.matmul(state, trainVariables[2]) + trainVariables[1] 
        # print("op_HiddenToHidden :  ", op_HiddenToHidden.shape)
        op_FromHidden = op_InpToHidden + op_HiddenToHidden
        # print("op_FromHidden :  ", op_FromHidden.shape)
        state = tf.nn.tanh(op_FromHidden)
        # print("op_FromHidden :  ", op_FromHidden.shape)


        # Second node
        op_FromOPLayer = tf.matmul(state, trainVariables[3]) + trainVariables[4]
        y = tf.nn.softmax(op_FromOPLayer)
        
        y = y.numpy()[0]

        


        seq.append(np.random.choice(vocab_size, p=y))

        char = [ind_to_ch[ind] for ind in seq]
    return sentence.join(char)
        
sentence = generate(2000)



In [0]:
print(sentence)

<S>s rade,
Thish hoc, anch, to, loth bo knows Pachas I ardard:
Anj lay ofFrink's thes who y I will cand arttinfs thes Menstyou they eaglong, sour wall,, that your profsend hald! und oof pain; Aod of youf lows; The, so hat thee, he loyd, Co with Puts foll, Lordus to of fou shreds sfow goch hegh thit thad gan:
Uot
I splich hof gilot?,
Tf As TmUS:
Mo, wo pod ey sting,
Dhd lay, and toodst whice, by So your. of to hiscoue wigh Ifot worde, the loves, my what thy dod. 's bik, wher you, bomes's bseid,
roue.
grat rowendourds: de;-yir upoamps, boow's go walll.

CAEN INA:
G staln, thotr uastery is for dord, yor uthin ssyid,
The batser hod sich, a rold, ond longh has. so prowho hold the eronnspran hes,
my merucerof in pose you, Mice of to soll.

CLEc:
Ptvis,,, ad I thes of woad you, By soune ruth Pucr oors igrd ho apray, lace urr herhour pooth you, harn?
Mh wIE: Guth y Goo.

BOoT:
Sivid sood, ond bet gods and..
CyENl:
Snclas theer--and not your garded foodst, bete's attislly dhbf,;e your duss?

fl

#Reference
1. https://github.com/vzhou842/rnn-from-scratch
2. http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/
3. https://www.analyticsvidhya.com/blog/2019/01/fundamentals-deep-learning-recurrent-neural-networks-scratch-python/